In [8]:
!cd ../ && pip install -e .

Obtaining file:///C:/dev/workspace/NYU/csgy6953_DeepLearning_Midterm
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for ResNetSrc (pyproject.toml): started
  Building editable for ResNetSrc (pyproject.toml): finished with status 'done'
  Created wheel for ResNetSrc: filename=ResNetSrc-0.0.1.dev0-0.editable-py3-none-any.whl size=6659 sha256=be85709492f044322140b5d327f07538f36748633fd661e7d5f1683bd27cac1f
  Stored in directory: C:\Users\alexa\AppData\Local\Temp\pip-ephem-wheel-cache-qsrw3f0p\wheels\9a\b8\06\09eddb28b116f9e70dce


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import time
import copy
import random

In [10]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_auto_transforms

BATCH_SIZE = 256
valid_ration = 0.1

train_iter, valid_iter, test_iter = (
    make_data_loaders(
        *get_transformed_data(
            make_transforms= make_auto_transforms,
            valid_ratio= valid_ration
        )
    )
)

ModuleNotFoundError: No module named 'src'

In [11]:
from src.model import ResNet, StemConfig, ResidualBlockType
# import src.model as models
from src.utils import initialize_parameters

arch = [
    (ResidualBlockType.BOTTLENECK, 2, 128, 0.1),
    (ResidualBlockType.BOTTLENECK, 6, 256, 0.1),
    (ResidualBlockType.BOTTLENECK, 6, 512, 0.1),
    (ResidualBlockType.BOTTLENECK, 2, 1024, 0.1),
]

config = StemConfig(num_channels=128, kernel_size=3, stride=1, padding=1)
model = ResNet(arch, stem_config=config, output_size=10, use_bias=True)

ModuleNotFoundError: No module named 'src'

In [13]:
print("Initializing new model")
inputs = torch.ones((BATCH_SIZE, 3, 32, 32))
model = model.to(device)
y = model(inputs.to(device))
print(y.size())

model.apply(initialize_parameters)

Initializing new model


NameError: name 'BATCH_SIZE' is not defined

In [14]:
import math

best_loss = float('inf')
EPOCHS = 3
max_lr = 3
start_lr = 0.0001

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=start_lr)

lr_lambda = lambda x: math.exp(x * math.log(max_lr / start_lr) / (EPOCHS * len(train_iter)))
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

NameError: name 'model' is not defined

In [ ]:
"https://towardsdatascience.com/adaptive-and-cyclical-learning-rates-using-pytorch-2bf904d18dee"

lr_stats = []
acc_stats = []
loss_stats = []

smoothing = 0.05
iter = 0
model.train()
for i in range(EPOCHS):
    for x,y in train_iter:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        outputs = model(x)
        loss = criterion(outputs, y)

        loss.backward()
        optimizer.step()

        scheduler.step()
        acc_stats.append((outputs.argmax(1) == y).sum().item())

        lr_stats.append(optimizer.state_dict()["param_groups"][0]["lr"])

        if iter == 0:
            loss_stats.append(loss.item())
        else:
            # apply smoothing
            smooth_loss = smoothing * loss.item() + (1 - smoothing) * loss_stats[-1]
            loss_stats.append(smooth_loss)

        iter += 1


In [ ]:
# add matplot lib graph